#### Lecture 1. RNN

1. RNN (Recurrent Neural Network)
- 이전 time의 hidden state 입력을 받아 현재의 정보를 결합하여 현재의 hidden state 생성
- 매 time step 마다 반복적으로 등장하는 모듈, output은 매타임 계산(번역) or 마지막만 계산(감정분석)
- 모델 diagram : rolled version / unrolled version
- $ h_t = f_W(h_{t-1}, x_t) $   << same function, same parameter at every time step
- $ h_t = tanh(W_{hh}h_{t-1} + W_{xh}x_t) $  , $y_t = W_{hy}h_t $

2. Types of RNN
- one to one : standard neural net(sequence, time step X) ex) 나이, 성별, 키로 혈압 범위 예측
- one to many : 입력은 단일 입력(첫번째 step에만 나머지는 제로 벡터로 채우게 됨), 출력만 time step   ex) 이미지로부터 글 생성
- many to one : 입력이 time step, 출력이 단순 출력(마지막 step에만)  ex) 감정 분석
- many to many : 입력과 출력이 모두 sequence
    - case 1) 입력을 다 읽고 출력 : machine translation
    - case 2) 입력과 동시에 출력 : video classfication, POS tagging

3. Character-level Language Model
- language model : 다음 단어나 문자열이 무엇인지 예측
- character-level : 각각의 문자열이 one-hot vector 형태로 입력으로 주어짐
- many to many 유형의 RNN에 해당 : $ Logit = W_{hy}h_t + b $
- 직전 time의 output 을 입력으로 하는 무한한 길이의 sequence 생성 가능
- sentence 의 경우 space( ), 구두점(.,), 줄바꿈도 character dict에 넣어서 학습
- 논문 작성, 마크다운 문서 작성에 쓰이는 Latex 역시 프로그래밍 언어, RNN 학습 가능

4. RNN 특징
- Backpropagation through time (BPTT)
    1. 전체 sequence 에 대해 forward, back
    2. sequence 중 chunk 에 대해 forward, back
    3. 전체 sequence 에 대해 forward, 그 중 일부만 back
- RNN의 정보를 저장하는 것은 결국 h_t : hidden state vector
    ex) quote detection cell, if statement cell
- banilla RNN : gradient exploding or vanishing 이 나타난다는 한계점

#### Lecture 2. LSTM and GRU

1. LSTM (Long Short-Term Memory)
- 별도의 transformation 없이 통과되는 cell state information 추가
- C<sub>t</sub> (cell state : 장기 memory에 해당), h<sub>t</sub> (hidden state : 단기 memory에 해당)
- $ \{C_t, h_t\} = LSTM(x_t, C_{t-1}, h_{t-1}) $
- i : input gate // f : forget gate // o : output gate // g : gate gate
    - forget gate : $ f_t = \sigma(W_f \cdot[h_{t-1}, x_t] + b_f) $
    - input gate : $ i_t = \sigma(W_i \cdot[h_{t-1}, x_t] + b_i) $
    - gate gate : $\tilde C_t = \tanh(W_c \cdot[h_{t-1}, x_t] + b_c) $
    - $C_t = f_t \cdot C_{t-1} + i_t \cdot\tilde C_t $
    - output gate : $o_t = \sigma(W_o \cdot[h_{t-1}, x_t] + b_o) $
    - $h_t = o_t \cdot tanh(C_t) $
- $C_t$ 는 기억해야되는 모든 정보를 담고 있는 벡터
- $h_t$ 는 현재 time step의 output layer의 입력으로 작용

2. Gated Recurrent Unit (GRU)
- LSTM에서 경량화 된 모델 (cell state가 존재하지 않으나 hidden state가 cell state 역할 수행)
- 하나의 gate controller $z_t$ 가 forget 과 input gate 모두 제어
- $ z_t = \sigma(W_z \cdot[h_{t-1}, x_t]) $
- $ r_t = \sigma(W_r \cdot[h_{t-1}, x_t]) $
- $ \tilde h_t = \tanh(W \cdot [r_t \cdot h_{t-1}, x_t]) $  gate gate 역할
- $ h_t = (1-z_t) \cdot h_{t-1} + z_t \cdot \tilde h_t $   가중평균
- 계산량과 메모리 양 감소, 성능이 LSTM과 비슷

- 필요한 정보를 곱셈이 아닌 덧셈으로 만들어 준다는 사실로부터 gradient vanishing 이 사라짐
- 덧셈 연산은 back prop 수행시 gradient를 복사해 주는 연산이 되므로 long term dependency 해결